# 03 — Sentiment Mapping (Text + Rating Combined)

In this notebook, we apply VADER sentiment to review text, generate rating-based sentiment,
and combine both into a final sentiment label.

✅ Input: `../data/netflix_reviews_clean.csv`  
✅ Output: `../data/netflix_reviews_cleaned.csv`


## Step 1: Load Clean Dataset

In [ ]:
import pandas as pd
from pathlib import Path

CLEAN_DATA_PATH = Path("../data/netflix_reviews_clean.csv")
df = pd.read_csv(CLEAN_DATA_PATH)

print(f"✅ Loaded cleaned dataset with {len(df)} reviews")
df.head()


## Step 2: Initialize VADER Sentiment Analyzer


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()


## Step 3: Sentiment from Text (VADER)



In [ ]:
# 1. VADER Sentiment Analysis & Categorization

def categorize_netflix_sentiment(score):
    if score >= 0.05:
        return "positive"
    elif score <= -0.05:
        return "negative"
    else:
        return "neutral"


In [ ]:
# 2. Apply VADER to each review
df["vader_score"] = df["clean_review"].apply(lambda x: sia.polarity_scores(str(x))["compound"])
df["sentiment"] = df["vader_score"].apply(categorize_netflix_sentiment)

## Step 4: Sentiment from Rating


In [ ]:
# 3. Sentiment from star ratings
# -----------------------------
def sentiment_from_rating(rating):
    if rating >= 4:   # 4 or 5 stars
        return "positive"
    elif rating == 3: # middle rating
        return "neutral"
    else:             # 1 or 2 stars
        return "negative"

In [ ]:
# 4. Apply to dataset
df["sentiment_rating"] = df["rating"].apply(sentiment_from_rating)


## Step 5: Combined Sentiment Logic


In [ ]:
# 5. Combined sentiment function
def combined_sentiment(vader, rating):
    if vader == rating:
        return vader  # Both agree
    elif vader == "neutral":
        return rating  # Trust rating if VADER is neutral
    elif rating == "neutral":
        return vader  # Trust VADER if rating is neutral
    else:
        # If one says positive and the other says negative → neutral
        return "neutral"


In [ ]:
# 6. Apply to dataset
df["sentiment_combined"] = df.apply(lambda x: combined_sentiment(x["sentiment"], x["sentiment_rating"]), axis=1)

## Step 6: Sentiment Alignment Analysis 

This section compares how VADER text sentiment aligns with user star-rating sentiment and analyzes how conflicts are resolved through the combined sentiment logic.


In [ ]:
# 7. Compare VADER vs Rating
# -----------------------------
comparison = (df["sentiment"] == df["sentiment_rating"]).mean()
print(f"✅ VADER matches rating-based sentiment {comparison*100:.2f}% of the time.")

In [ ]:
# 8. Comapre Sentiment Combined vs Rating
comparison = (df["sentiment_combined"] == df["sentiment_rating"]).mean()
print(f"✅ VADER matches rating-based sentiment {comparison*100:.2f}% of the time.")

## Step 7: Summary Statistics 

In [ ]:
# 9. Summary statistics
print("\n📊 Sentiment Summary")
print(df['sentiment'].value_counts())

In [ ]:
# 10. Summary statistics for rating-based sentiment
print("\n📊 Rating-based Sentiment Summary")
print(df['sentiment_rating'].value_counts())

In [ ]:
# 11. Summary statistics for combined sentiment
print("\n📊 Combined Sentiment Summary")
print(df['sentiment_combined'].value_counts())

## 💾 Step 6: Save Cleaned Dataset


In [ ]:
# 12. Save Cleaned dataset
CLEANED_OUTPUT_LABELED_PATH = Path("../data/netflix_reviews_Cleaned.csv")
df.to_csv(CLEANED_OUTPUT_LABELED_PATH, index=False)

print("\n✅ Sentiment-labeled dataset saved to 'data/netflix_reviews_Cleaned.csv'")
print(f"📊 Final labeled reviews: {len(df)}")
df.head()


### Sentiment Labeling Completed!
➡️ Next Notebook: **04_exploratory_data_analysis.ipynb**
